# 🎮 Fantasy Football Database - Master Control Panel

This notebook provides complete control over your fantasy football projections database.

**Features:**
- 🗑️ Database management (view, clear, reset)
- 📊 Run scrapers individually or all at once
- 📈 View data quality metrics
- 🔍 Check team coverage by source


## 📦 Setup & Imports

**Note:** The FanDuel scraper (Playwright-based) runs via subprocess to avoid conflicts with Jupyter's event loop. All other scrapers run directly in the notebook.


In [1]:
import os
import sqlite3
import subprocess
import sys
from datetime import datetime
from database import ProjectionsDB

# Don't import Playwright-based scrapers directly - they conflict with Jupyter
# We'll import Selenium-based scrapers normally
from scraper_sleeper import SleeperScraper
from scraper_espn import ESPNScraper
from scraper_fantasypros import FantasyProsScraper
from scraper_firstdown import FirstDownStudioScraper

# Configuration
WEEK = "Week 9"
SEASON = "2024"
HEADLESS = True  # Set to False to see browser

print(f"✓ Imports successful")
print(f"Configuration: {WEEK}, Season {SEASON}")
print(f"Headless mode: {HEADLESS}")
print(f"\n⚠️  Note: FanDuel scraper must be run via subprocess (Playwright limitation in Jupyter)")


✓ Imports successful
Configuration: Week 9, Season 2024
Headless mode: True

⚠️  Note: FanDuel scraper must be run via subprocess (Playwright limitation in Jupyter)


## 📊 Database Status & Overview


In [5]:
def show_database_status():
    """Display current database status."""
    with ProjectionsDB() as db:
        all_projs = db.get_projections()
        
        print(f"\n{'='*70}")
        print(f"DATABASE STATUS")
        print(f"{'='*70}\n")
        print(f"Total records: {len(all_projs)}")
        
        # Count by source
        print(f"\n📊 Records by Source:")
        print("-" * 70)
        sources = {}
        for p in all_projs:
            source = p['source_website']
            sources[source] = sources.get(source, 0) + 1
        
        for source, count in sorted(sources.items()):
            print(f"  {source:<25} {count:>5} records")
        
        # Count by week
        print(f"\n📅 Records by Week:")
        print("-" * 70)
        weeks = {}
        for p in all_projs:
            week = p['week']
            weeks[week] = weeks.get(week, 0) + 1
        
        for week, count in sorted(weeks.items()):
            print(f"  {week:<25} {count:>5} records")
        
        # Team coverage
        print(f"\n🏈 Team Data Coverage:")
        print("-" * 70)
        for source in sorted(sources.keys()):
            source_projs = [p for p in all_projs if p['source_website'] == source]
            with_team = sum(1 for p in source_projs if p.get('team'))
            total = len(source_projs)
            pct = (with_team / total * 100) if total > 0 else 0
            status = "✅" if pct >= 95 else "⚠️" if pct >= 75 else "❌"
            print(f"  {source:<25} {with_team:>4}/{total:<4} ({pct:>5.1f}%) {status}")

show_database_status()



DATABASE STATUS

Total records: 804

📊 Records by Source:
----------------------------------------------------------------------
  espn.com                    233 records
  fantasypros.com             182 records
  sleeper.com                 389 records

📅 Records by Week:
----------------------------------------------------------------------
  Week 8                      182 records
  Week 9                      622 records

🏈 Team Data Coverage:
----------------------------------------------------------------------
  espn.com                   233/233  (100.0%) ✅
  fantasypros.com            182/182  (100.0%) ✅
  sleeper.com                389/389  (100.0%) ✅


## 🗑️ Database Management

### ⚠️ Clear Entire Database
**WARNING:** This will delete ALL records from the database!


In [ ]:
def clear_entire_database():
    """Delete all records from the database."""
    confirm = input("⚠️  Are you sure you want to delete ALL records? Type 'DELETE ALL' to confirm: ")
    
    if confirm == "DELETE ALL":
        conn = sqlite3.connect("projections.db")
        cursor = conn.cursor()
        cursor.execute("DELETE FROM projections")
        conn.commit()
        deleted = cursor.rowcount
        conn.close()
        print(f"\n✓ Deleted {deleted} records from database")
        print("Database is now empty.")
    else:
        print("\n✗ Cancelled. Database unchanged.")

# Uncomment to run:
clear_entire_database()
show_database_status()


### 🗑️ Clear Specific Source


In [4]:
def clear_source(source_name, week=None):
    """Delete records from a specific source."""
    conn = sqlite3.connect("projections.db")
    cursor = conn.cursor()
    
    if week:
        cursor.execute("DELETE FROM projections WHERE source_website = ? AND week = ?", (source_name, week))
        print(f"✓ Deleted {cursor.rowcount} records from {source_name} for {week}")
    else:
        cursor.execute("DELETE FROM projections WHERE source_website = ?", (source_name,))
        print(f"✓ Deleted {cursor.rowcount} records from {source_name} (all weeks)")
    
    conn.commit()
    conn.close()

# Examples:
# clear_source("fantasypros.com", WEEK)
clear_source("fanduel.com")
# show_database_status()


✓ Deleted 465 records from fanduel.com (all weeks)


## 🤖 Run Scrapers Individually

**Note:** FanDuel uses Playwright which conflicts with Jupyter's event loop, so it runs via subprocess (separate process). The other scrapers run directly in the notebook.

### 1️⃣ FanDuel Scraper


In [6]:
print(f"\n{'='*70}")
print(f"🔵 FANDUEL SCRAPER (via subprocess)")
print(f"{'='*70}\n")
print(f"Using configuration: {WEEK}, Headless={HEADLESS}\n")

# FanDuel uses Playwright which doesn't work in Jupyter's event loop
# Run it as a subprocess instead
try:
    # Create a temporary script to run FanDuel scraper
    # Use proper string formatting to pass variables
    script_content = f'''from scraper_fanduel import FanDuelScraper

print("Starting FanDuel scraper...")
print(f"Week: {WEEK}")
print(f"Headless: {HEADLESS}")

with FanDuelScraper(headless={HEADLESS}) as scraper:
    scraper.scrape_and_save(week="{WEEK}")
'''
    
    # Write to temp file
    with open("_temp_fanduel.py", "w") as f:
        f.write(script_content)
    
    # Debug: Show what we're running
    print(f"Running FanDuel scraper for {WEEK}...\n")
    
    # Run as subprocess
    result = subprocess.run(
        [sys.executable, "_temp_fanduel.py"],
        capture_output=True,
        text=True
    )
    
    # Print output
    print(result.stdout)
    
    if result.returncode == 0:
        print("\n✅ FanDuel scraping complete!")
    else:
        print(f"\n❌ Error: {result.stderr}")
    
    # Clean up temp file
    if os.path.exists("_temp_fanduel.py"):
        os.remove("_temp_fanduel.py")
        
except Exception as e:
    print(f"\n❌ Error: {e}")
    # Clean up temp file if error
    if os.path.exists("_temp_fanduel.py"):
        os.remove("_temp_fanduel.py")

show_database_status()



🔵 FANDUEL SCRAPER (via subprocess)

Using configuration: Week 9, Headless=True

Running FanDuel scraper for Week 9...

Starting FanDuel scraper...
Week: Week 9
Headless: True
Navigating to https://www.fanduel.com/research/nfl/fantasy/ppr...
  Intercepted data from https://fdresearch-api.fanduel.com/graphql
  âœ“ Found 465 projections in response

Parsing 465 projections...
  Lamar Jackson (QB): 24.7 pts
  Christian McCaffrey (RB): 22.8 pts
  Josh Allen (QB): 21.6 pts
  Jonathan Taylor (RB): 21.3 pts
  Patrick Mahomes (QB): 21.0 pts
  Kyler Murray (QB): 20.8 pts
  Jahmyr Gibbs (RB): 20.6 pts
  Justin Herbert (QB): 20.6 pts
  Drake Maye (QB): 20.5 pts
  Puka Nacua (WR): 20.0 pts
  Ja'Marr Chase (WR): 19.8 pts
  Dak Prescott (QB): 19.7 pts
  Bijan Robinson (RB): 19.5 pts
  Jordan Love (QB): 19.3 pts
  Caleb Williams (QB): 19.2 pts
  De'Von Achane (RB): 19.1 pts
  CeeDee Lamb (WR): 18.5 pts
  James Cook (RB): 18.2 pts
  Jaxson Dart (QB): 18.2 pts
  Zay Flowers (WR): 18.1 pts
  Trey McBrid

### 2️⃣ Sleeper Scraper


In [ ]:
print(f"\n{'='*70}")
print(f"💤 SLEEPER SCRAPER")
print(f"{'='*70}\n")

try:
    with SleeperScraper() as scraper:
        scraper.scrape_and_save(week=WEEK, season=SEASON)
    print("\n✅ Sleeper scraping complete!")
except Exception as e:
    print(f"\n❌ Error: {e}")

show_database_status()


### 3️⃣ ESPN Scraper


In [ ]:
print(f"\n{'='*70}")
print(f"🔴 ESPN SCRAPER")
print(f"{'='*70}\n")

try:
    with ESPNScraper(headless=HEADLESS) as scraper:
        scraper.scrape_and_save(week=WEEK, season=SEASON)
    print("\n✅ ESPN scraping complete!")
except Exception as e:
    print(f"\n❌ Error: {e}")

show_database_status()


### 4️⃣ FantasyPros Scraper


In [ ]:
print(f"\n{'='*70}")
print(f"📊 FANTASYPROS SCRAPER")
print(f"{'='*70}\n")

try:
    with FantasyProsScraper(headless=HEADLESS) as scraper:
        scraper.scrape_and_save(week=WEEK)
    print("\n✅ FantasyPros scraping complete!")
except Exception as e:
    print(f"\n❌ Error: {e}")

show_database_status()


### 5️⃣ First Down Studio Scraper


In [ ]:
print(f"\n{'='*70}")
print(f"🎯 FIRST DOWN STUDIO SCRAPER")
print(f"{'='*70}\n")

try:
    with FirstDownStudioScraper(headless=HEADLESS) as scraper:
        scraper.scrape_and_save(week=WEEK, scoring="PPR")
    print("\n✅ First Down Studio scraping complete!")
except Exception as e:
    print(f"\n❌ Error: {e}")

show_database_status()


## 🎯 Quick Reference

### Useful Commands

```python
# View current database status
show_database_status()

# Clear entire database (requires confirmation)
clear_entire_database()

# Clear specific source
clear_source("fantasypros.com", WEEK)  # Clear specific week
clear_source("fanduel.com")            # Clear all weeks

# Change configuration
WEEK = "Week 9"
SEASON = "2024"
HEADLESS = False  # Show browser
```

### Source Names
- `"fanduel.com"`
- `"sleeper.com"`
- `"espn.com"`
- `"fantasypros.com"`
- `"firstdown.studio"`
